## Pressure sensor placement for leakage detection


### Initialize EPANET Python Toolkit (EPyT)

You should always begin with this command to import the toolkit.

[EPyT](https://github.com/OpenWaterAnalytics/EPyT) is available on [PyPI](https://pypi.org/project/epyt/) and can be installed via `pip install epyt`. To upgrade to the latest version if it's already installed, use `pip install --upgrade epyt`.

In [ ]:
%pip install epyt pymoo

In [ ]:
%pip install numpy==1.23

### Download data

In [ ]:
import requests

url = "https://raw.githubusercontent.com/WaterFutures/EPyT-and-EPyT-Flow-Tutorial/main/data/Hanoi.inp?token=GHSAT0AAAAAACTQ7OWEQMMHP5BC7ZRAI5JKZT5MS7A"
file_path = "Hanoi.inp"

try:
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded and saved as {file_path}")
except requests.exceptions.HTTPError as err:
    print(f"HTTP error occurred: {err}")
except Exception as err:
    print(f"An error occurred: {err}")


In [ ]:
from epyt import epanet

import numpy as np

from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SimulatedBinaryCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
import matplotlib.pyplot as plt

### Water Network

In [ ]:
# Load the water network model
d=epanet('Hanoi.inp')

In [ ]:
# Get the number of junctions and nodes
nj = d.getNodeJunctionCount()
nn = d.getNodeCount()
d.printv(nj)
d.printv(nn)

In [ ]:
reservoir_indices = d.getNodeReservoirIndex()
junction_indices = d.getNodeJunctionIndex()
junction_indices_py = np.array(junction_indices) - 1

d.printv(reservoir_indices)
d.printv(junction_indices_py) 

### Calculate Sensitivity matrix
[1] Cugueró , M., Puig, V., Quevedo, J. Optimal pressure sensor placement and assessment for leak location using a relaxed isolation index: Application to the Barcelona water network, Control Eng. Pract., vol. 63, pp. 1-12, Jun. 2017.

##### Calculate healthy states in extended time simulation

In [ ]:
d.setTimeSimulationDuration(24 * 60 * 60)  # greater weight to low demand hours
d.setTimePatternStart(0)  # in seconds

allParameters = d.getComputedTimeSeries()
P0 = allParameters.Pressure[:, junction_indices_py].T
Dem0 = allParameters.Demand[:, junction_indices_py].T
simSteps = P0.shape[1]

# Create Augmented-time Sensitivity Matrix
# Simulate all leakages and get all scenarios pressures
leak_mag_desir = np.mean(np.mean(Dem0[Dem0 > 0]))
mean_pressure = np.mean(np.mean(P0[P0 > 0]))

leak_emit = leak_mag_desir / np.sqrt(mean_pressure)
emit0 = d.getNodeEmitterCoeff()
S = np.zeros((nj, nj))

for leak in range(1, nj + 1):
    print('Calculating Sensitivity Matrix...')
    print('Simulating leakage', leak, 'out of', nj)
    emit = np.zeros_like(emit0)
    emit[leak - 1] = leak_emit  # set emitter coefficient (leakage) value
    d.setNodeEmitterCoeff(emit)

    allParameters = d.getComputedTimeSeries()

    P = allParameters.Pressure[:, junction_indices_py].T
    Dem = allParameters.Demand[:, junction_indices_py].T
    leak_mag = Dem[leak - 1, :] - Dem0[leak - 1, :]
    Stmp = (P - P0) / leak_mag
    rmax = np.max(np.abs(Stmp), axis=0)
    Stmp = np.abs(Stmp) / rmax
    S[:, leak - 1] = np.max(Stmp, axis=1)

print('Calculations done!')


### Place sensors using GA
Select number of sensors to be installed:

In [ ]:
sensors_num = 4

Define existing sensors in the network

In [ ]:
exist_sens_ind = []; # Define existing sensors in the network
# exist_sens_ind=d.getNodeIndex({'14','63','114','399','572','302','662'});

##### Solve using GA: Maximize the minimum sensitivity of all sensors to any leakage

In [ ]:
#### Sensor Placement optimization problem - Cost function

In [ ]:
# Cost function: min sensitivity of all leaks
def GACostFunction(u, S):
    Sm = S[u > 0, :]
    Smax = np.max(Sm, axis=0)
    # cost = 1-np.min(Smax)-0.1*np.mean(Smax)
    # cost = 1-np.min(Smax)
    cost = 1-np.max(Smax)
    return cost
    
# Sensor placement optimization problem for a genetic algorithm. It sets up the problem
# dimensions, variable bounds, and defines how to evaluate each solution's fitness using
# the provided cost function (GACostFunction). The genetic algorithm can then use this
# problem definition to evolve sensor placement configurations towards an optimal solution.
class SensorPlacementProblem(Problem):
    def __init__(self, S, nj, sensors_num, exist_sens_ind):
        self.S = S
        self.nj = nj
        self.sensors_num = sensors_num
        self.exist_sens_ind = exist_sens_ind
        super().__init__(n_var=nj, n_obj=1, n_constr=0, xl=0, xu=1, type_var=int)

    def _evaluate(self, x, out, *args, **kwargs):
        f = np.array([GACostFunction(xi, self.S) for xi in x])
        out["F"] = f

In [ ]:
problem = SensorPlacementProblem(S, nj, sensors_num, exist_sens_ind)

In [ ]:
# Configure the Genetic Algorithm (GA)
algorithm = GA(
    pop_size=200,  # Population size
    sampling=IntegerRandomSampling(),  # Correctly specified sampling method
    crossover=SimulatedBinaryCrossover(eta=20, prob=1.0),  # Correctly specified crossover method
    mutation=PolynomialMutation(eta=20, prob=1.0 / nj)  # Correctly specified mutation method
)

In [ ]:
# Solve the optimization problem
res = minimize(problem,
               algorithm,
               ('n_gen', 100),
               #  multiple criteria (x_tol for tolerance, n_gen for maximum generations,
               #  and n_last for stall generations - terminates if there is no improvement
               #  in the last 5 generations)
               seed=1,  # The seed=1 is used to ensure reproducibility of results.
               verbose=True)  # flag is included to provide detailed output during the optimization process.
# Extract the result
u = res.X

#### Plot sensors

In [ ]:
# Get indices of top sensors
top_sensors_indices = np.argsort(u)[-sensors_num:][::-1]
senors_ids = d.getNodeNameID(list(top_sensors_indices + 1))
# print("Indices of top", sensors_num, "selected sensors:", senors_ids)

# Load optimal solution (for demonstration only!)
sens_IDs_demonstration = ['13', '16', '22', '30']
sens_ind = d.getNodeIndex(sens_IDs_demonstration)


def custom_plot(coor):
    x_nodes = list(coor['x'].values())
    y_nodes = list(coor['y'].values())
    # Get links connecting nodes
    links = d.getNodesConnectingLinksIndex()
    # Plot the network manually
    fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
    for link in links:
        x_coords = [coor['x'][link[0]], coor['x'][link[1]]]
        y_coords = [coor['y'][link[0]], coor['y'][link[1]]]
        ax.plot(x_coords, y_coords, color='steelblue', linewidth=0.7)
    ax.axis('off')
    # Plot nodes
    ax.scatter(x_nodes, y_nodes, color='blue', s=10, zorder=5)
    return ax


# Get node coordinates
coor = d.getNodeCoordinates()
# Custom plot - network plot
ax = custom_plot(coor)

# Plot sensors on the network
for sens in sens_ind:
    # Add plot point for the leak node
    x_sens = coor['x'][sens]
    y_sens = coor['y'][sens]

    sens_id = d.getNodeNameID(sens)
    # Plot the sens node with a higher zorder
    ax.plot(x_sens, y_sens, 'o', color='r', markersize=12, zorder=10)
    ax.text(x_sens + 60, y_sens, sens_id, color='black', fontweight='bold', fontsize=11, zorder=10)

# Set the title and show the plot
ax.set_title(f'Number of sensors: {len(sens_ind)}', fontweight='bold')
plt.show()